In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [5]:
reviews = pd.read_csv('reviews.txt',header=None)

In [6]:
reviews.columns

Int64Index([0], dtype='int64')

In [7]:
labels = pd.read_csv('labels.txt',header=None)

In [8]:
from collections import Counter

In [9]:
total_counts = Counter()

In [10]:
for _, row in reviews.iterrows():
    total_counts.update(row[0].split(' '))
print("Total words ", len(total_counts))

Total words  74074


In [11]:
vocab = sorted(total_counts, key=total_counts.get, reverse=True)[:10000]

In [12]:
vocab[-1]

'hackenstein'

In [13]:
total_counts[vocab[-1]]

30

In [14]:
reviews.shape

(25000, 1)

In [15]:
word2idx = {word: i for i, word in enumerate(vocab)}

In [22]:
word2idx

{'': 0,
 'sant': 9484,
 'going': 170,
 'report': 4414,
 'goodness': 4063,
 'gray': 3654,
 'how': 88,
 'pegg': 6430,
 'sandy': 8069,
 'painted': 4198,
 'groups': 3997,
 'rudd': 8569,
 'image': 1438,
 'carla': 4819,
 'holocaust': 5861,
 'gossip': 9925,
 'filmed': 797,
 'abortion': 7095,
 'gave': 511,
 'magazine': 3328,
 'regardless': 3542,
 'fat': 1888,
 'antagonist': 7123,
 'automatic': 7913,
 'stunned': 5017,
 'hardships': 9800,
 'icy': 8595,
 'revelation': 3384,
 'opposition': 8372,
 'stares': 8313,
 'shoots': 3221,
 'ritual': 6703,
 'archie': 9843,
 'realized': 1679,
 'wang': 5024,
 'superman': 1714,
 'kerry': 9799,
 'gambling': 5410,
 'galactica': 6990,
 'with': 18,
 'sensitivity': 8098,
 'offside': 9943,
 'packs': 6394,
 'bend': 5824,
 'crack': 4053,
 'international': 1947,
 'pleasing': 5677,
 'movements': 3792,
 'roberts': 3219,
 'corrupt': 3386,
 'literary': 5269,
 'reluctantly': 7634,
 'senator': 7805,
 'history': 468,
 'lugosi': 2512,
 'raunchy': 7702,
 'hatred': 3664,
 'promot

In [16]:

def text_to_vector(text):
    word_vector = np.zeros(len(vocab), dtype=np.int_)
    for word in text.split(' '):
        idx = word2idx.get(word, None)
        if idx is None:
            continue
        else:
            word_vector[idx] += 1
    return np.array(word_vector)

In [17]:
text_to_vector('The tea is for a party to celebrate how how').shape

(10000,)

In [18]:
word_vectors = np.zeros((len(reviews), len(vocab)), dtype=np.int_)

In [19]:
word_vectors.shape

(25000, 10000)

In [20]:
for ii, (_,text) in enumerate(reviews.iterrows()):
    word_vectors[ii] = text_to_vector(text[0])

In [21]:
Y = (labels == 'positive').astype(np.int_)

In [25]:
shuffle = np.arange(len(labels))
np.random.shuffle(shuffle)
test_fraction = 0.9
label_length = len(labels)
train_split, test_split = shuffle[:int(label_length*test_fraction)], shuffle[int(label_length*test_fraction):]

In [31]:
trainX = word_vectors[train_split,:]

In [33]:
trainX.shape

(22500, 10000)

In [34]:
trainY = to_categorical(Y.values[train_split],2)

In [38]:
Y.values[test_split]

array([[1],
       [1],
       [0],
       ..., 
       [0],
       [1],
       [1]])

In [36]:
def build_model():
    tf.reset_default_graph()
    
    #Input
    net = tflearn.input_data([None, 10000])
    
    #Hidden Layers
    net = tflearn.fully_connected(net, 200, activation='ReLU')
    net = tflearn.fully_connected(net, 25,activation='ReLU')
    
    #Output Layer
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

In [37]:
model = build_model()

In [ ]:
#model.fit(word_vectors, Y, show_metric=True, batch_size=128, n_epoch=100)